In [1]:
# load env with api keys https://stackoverflow.com/a/54028874
%load_ext dotenv
%dotenv ../etc/config.env



In [2]:
import sys
sys.path.append("../")

import os
import requests
from urllib.parse import urlparse
import json
import pandas as pd
from mastodon import Mastodon

from desci_sense.schema.post import RefPost
from desci_sense.dataloaders.mastodon.mastodon_loader import MastodonLoader

In [3]:
mloader = MastodonLoader()

In [6]:
accts = ["@ronent@mastodon.social",'@jrashf@mastodonapp.uk']
posts = mloader.load_profiles(mastodon_accounts=accts)

In [7]:
posts

[RefPost(author='Ronen Tamari', content='', url='https://mastodon.social/users/ronent/statuses/111411889770523707/activity', metadata={'id': 111411889770523707, 'created_at': datetime.datetime(2023, 11, 15, 1, 8, 45, 66000, tzinfo=tzutc()), 'in_reply_to_id': None, 'in_reply_to_account_id': None, 'sensitive': False, 'spoiler_text': '', 'visibility': 'public', 'language': None, 'uri': 'https://mastodon.social/users/ronent/statuses/111411889770523707/activity', 'url': 'https://mastodon.social/users/ronent/statuses/111411889770523707/activity', 'replies_count': 0, 'reblogs_count': 0, 'favourites_count': 0, 'edited_at': None, 'favourited': False, 'reblogged': False, 'muted': False, 'bookmarked': False, 'content': '', 'filtered': [], 'reblog': {'id': 111410591996456032, 'created_at': datetime.datetime(2023, 11, 14, 19, 38, 42, 608000, tzinfo=tzutc()), 'in_reply_to_id': None, 'in_reply_to_account_id': None, 'sensitive': False, 'spoiler_text': '', 'visibility': 'public', 'language': 'en', 'uri

In [8]:
from langchain.document_loaders import MastodonTootsLoader

In [ ]:
loader = MastodonTootsLoader(
    mastodon_accounts=accts,
    number_toots=5,  # Default value is 100
)

In [3]:
p = Post(author="f",
     content="f",
     url="dd")

In [4]:
mp = MentionPost(author="f",
     content="f",
     url="dd",
     ref_url=["vv"])
mp

MentionPost(author='f', content='f', url='dd', ref_url=['vv'])

In [5]:
mp.source_network

'unkown'

In [3]:
mastodon = Mastodon(
        api_base_url= 'https://mastodon.social',
        access_token = os.environ["MASTO_ACCESS_TOKEN"]
    )

In [4]:
# Get user ID from username
user_id = mastodon.search(q='@jrashf@mastodonapp.uk')['accounts'][0]['id']

In [5]:
user_id

109309183510737296

In [8]:
# Get user's statuses (posts)
user_posts = mastodon.account_statuses(user_id, limit=5)

In [9]:
user_posts

[{'id': 111359565228905217,
  'created_at': datetime.datetime(2023, 11, 5, 19, 21, 52, tzinfo=tzutc()),
  'in_reply_to_id': 111358188921865048,
  'in_reply_to_account_id': 109240050397113639,
  'sensitive': False,
  'spoiler_text': '',
  'visibility': 'public',
  'language': 'en',
  'uri': 'https://mastodonapp.uk/users/jrashf/statuses/111359565033795338',
  'url': 'https://mastodonapp.uk/@jrashf/111359565033795338',
  'replies_count': 0,
  'reblogs_count': 0,
  'favourites_count': 0,
  'edited_at': None,
  'favourited': False,
  'reblogged': False,
  'muted': False,
  'bookmarked': False,
  'content': '<p><span class="h-card"><a href="https://tecnfy.com/@andy" class="u-url mention" rel="nofollow noopener noreferrer" target="_blank">@<span>andy</span></a></span> Thanks, I\'ll give this ago</p>',
  'filtered': [],
  'reblog': None,
  'account': {'id': 109309183510737296,
   'username': 'jrashf',
   'acct': 'jrashf@mastodonapp.uk',
   'display_name': 'James Ashford',
   'locked': False,
 

In [7]:
acct = '@jrashf@mastodonapp.uk'
max_posts = 10

In [4]:
def user_lookup(acct):
    URL = f'https://mastodon.social/api/v1/accounts/lookup'
    params = {
        'acct': acct
    }

    r = requests.get(URL, params=params)
    user = json.loads(r.text)
    
    return user

In [11]:
user = user_lookup(acct=acct)
user_id = user['id']

URL = f'https://mastodon.social/api/v1/accounts/{user_id}/statuses'
params = {
    'limit': 40
}

results = []

while len(results) <= max_posts:
    r = requests.get(URL, params=params)
    toots = json.loads(r.text)
    print(toots)
    if len(toots) == 0:
        break
    
    results.extend(toots)
    
    max_id = toots[-1]['id']
    params['max_id'] = max_id
    
df = pd.DataFrame(results)

[{'id': '111359565228905217', 'created_at': '2023-11-05T19:21:52.000Z', 'in_reply_to_id': '111358188921865048', 'in_reply_to_account_id': '109240050397113639', 'sensitive': False, 'spoiler_text': '', 'visibility': 'public', 'language': 'en', 'uri': 'https://mastodonapp.uk/users/jrashf/statuses/111359565033795338', 'url': 'https://mastodonapp.uk/@jrashf/111359565033795338', 'replies_count': 0, 'reblogs_count': 0, 'favourites_count': 0, 'edited_at': None, 'content': '<p><span class="h-card"><a href="https://tecnfy.com/@andy" class="u-url mention" rel="nofollow noopener noreferrer" target="_blank">@<span>andy</span></a></span> Thanks, I\'ll give this ago</p>', 'reblog': None, 'account': {'id': '109309183510737296', 'username': 'jrashf', 'acct': 'jrashf@mastodonapp.uk', 'display_name': 'James Ashford', 'locked': False, 'bot': False, 'discoverable': True, 'group': False, 'created_at': '2022-11-07T00:00:00.000Z', 'note': '<p>28, <a href="https://mastodonapp.uk/tags/Christian" class="mention 

In [13]:
len(toots)

40

In [14]:
toots[0]

{'id': '111359565228905217',
 'created_at': '2023-11-05T19:21:52.000Z',
 'in_reply_to_id': '111358188921865048',
 'in_reply_to_account_id': '109240050397113639',
 'sensitive': False,
 'spoiler_text': '',
 'visibility': 'public',
 'language': 'en',
 'uri': 'https://mastodonapp.uk/users/jrashf/statuses/111359565033795338',
 'url': 'https://mastodonapp.uk/@jrashf/111359565033795338',
 'replies_count': 0,
 'reblogs_count': 0,
 'favourites_count': 0,
 'edited_at': None,
 'content': '<p><span class="h-card"><a href="https://tecnfy.com/@andy" class="u-url mention" rel="nofollow noopener noreferrer" target="_blank">@<span>andy</span></a></span> Thanks, I\'ll give this ago</p>',
 'reblog': None,
 'account': {'id': '109309183510737296',
  'username': 'jrashf',
  'acct': 'jrashf@mastodonapp.uk',
  'display_name': 'James Ashford',
  'locked': False,
  'bot': False,
  'discoverable': True,
  'group': False,
  'created_at': '2022-11-07T00:00:00.000Z',
  'note': '<p>28, <a href="https://mastodonapp.u

In [6]:
df

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,edited_at,content,reblog,account,media_attachments,mentions,tags,emojis,card,poll
0,111359565228905217,2023-11-05T19:21:52.000Z,111358188921865048,109240050397113639,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/111359565033795338,...,None,"<p><span class=""h-card""><a href=""https://tecnf...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '109240050397113639', 'username': 'and...",[],[],None,None
1,111357290614377395,2023-11-05T09:43:27.000Z,111357253509524656,110805131752102725,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/111357290590134607,...,None,"<p><span class=""h-card""><a href=""https://masto...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '110805131752102725', 'username': 'dio...",[],[],None,None
2,111357215622513735,2023-11-05T09:24:23.000Z,None,None,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/111357215601924666,...,None,<p>Anyone else having much luck installing <a ...,None,"{'id': '109309183510737296', 'username': 'jras...",[],[],"[{'name': 'mastodon', 'url': 'https://mastodon...",[],None,None
3,111357202113595176,2023-11-05T09:20:57.000Z,111353506550101830,109350324884666820,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/111357202092534573,...,None,"<p><span class=""h-card""><a href=""https://masto...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '109350324884666820', 'username': 'syl...",[],[],None,None
4,111340699845217005,2023-11-02T11:24:12.000Z,110207898906665236,1251113,False,,unlisted,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/111340699816479124,...,None,"<p><span class=""h-card""><a href=""https://fedis...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '1251113', 'username': 'tfardet', 'url...",[],[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,109757688267982944,2023-01-26T21:41:56.000Z,109757355923384924,109540798563216101,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/109757681915996705,...,None,"<p><span class=""h-card""><a href=""https://mstdn...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '109540798563216101', 'username': 'jen...",[],[],None,None
153,109726573124491008,2023-01-21T09:50:33.000Z,109722765036019463,109582044164301105,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/109726573079757247,...,None,"<p><span class=""h-card""><a href=""https://masto...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '109582044164301105', 'username': 'Jus...",[],[],None,None
154,109710314864153685,2023-01-18T12:55:52.000Z,None,None,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/109710314840007152,...,None,<p>Who says you can’t have a full English for ...,None,"{'id': '109309183510737296', 'username': 'jras...","[{'id': '110136149368822552', 'type': 'image',...",[],[],[],None,None
155,109683040035581932,2023-01-13T17:17:45.000Z,109683023840109998,109348502229543855,False,,public,en,https://mastodonapp.uk/users/jrashf/statuses/1...,https://mastodonapp.uk/@jrashf/109683033085143481,...,None,"<p><span class=""h-card""><a href=""https://masto...",None,"{'id': '109309183510737296', 'username': 'jras...",[],"[{'id': '109348502229543855', 'username': 'woo...",[],[],None,None


In [3]:
test_post = "https://mastodon.social/@Iris@scholar.social/111269570161082863"